Extract tickers obtaineed from ssm.ipynb

In [2]:
from pprint import pprint

ticker_names = []

with open(r'./subset_c.txt', 'r') as fp:
    for line in fp:
        x = line[:-1] # excluding \n char
        ticker_names.append(x)

pprint(ticker_names)

['ADI',
 'EPD',
 'DPZ',
 'MTG',
 'DHR',
 'PRGO',
 'FTV',
 'AEO',
 'LHX',
 'CDP',
 'ROIC',
 'REG',
 'LFUS',
 'TSM',
 'ODFL',
 'VTR',
 'VLO',
 'MOS',
 'CF',
 'IVZ',
 'CSL',
 'BG',
 'TGNA',
 'MGY',
 'GLW',
 'J',
 'BABA',
 'INVH',
 'ROP',
 'GIL',
 'FAST',
 'DGX',
 'WD',
 'PARAA',
 'PSA',
 'AME',
 'JCI',
 'BX',
 'OVV',
 'IVT',
 'SWKS',
 'DBRG',
 'ADM',
 'KMI',
 'GRMN',
 'VTRS',
 'CSCO',
 'COP',
 'SPGI',
 'PFE',
 'MSFT',
 'DLR',
 'OXY',
 'SHOO',
 'WPC',
 'LCII',
 'GPN',
 'KIM',
 'XOM',
 'KDP',
 'MAA',
 'EOG',
 'KHC',
 'AOS',
 'MSM',
 'TAP',
 'ATO',
 'X',
 'IR',
 'IEX',
 'HLNE',
 'THO',
 'CNXC',
 'MKTX',
 'TXN',
 'OWL',
 'ALG',
 'PXD',
 'RHI',
 'MDC',
 'DLB',
 'MATX',
 'PSX',
 'TOL',
 'FLO',
 'EQR',
 'HRL',
 'CEIX',
 'A',
 'LKQ',
 'RYN',
 'ICFI',
 'CTRA',
 'JNJ',
 'CRC',
 'RMR',
 'PK',
 'SCCO',
 'HLI',
 'STLD',
 'PKG',
 'DHI',
 'DIS',
 'CHRD',
 'GOLF',
 'CTO',
 'CIVI',
 'BAM',
 'PEAK',
 'FIS',
 'WSO',
 'DTM',
 'WOR',
 'MRK',
 'KRC',
 'ACN',
 'DVN',
 'FANG',
 'SBRA',
 'PHM',
 'LPX',
 'ITIC',
 

Rank the target stock list by descending dividend yield.

In [3]:
import yfinance as yf

tickers = list(yf.Tickers(ticker_names).tickers.values())

# pprint(tickers[0].info)
cnt = 0
for ticker in tickers:
    try:
        ticker.info['dividendYield']
    except KeyError:
        tickers.remove(ticker)

sorted_tickers = sorted(tickers, key=lambda x: x.info["dividendYield"], reverse=True)
print(sorted_tickers[0].info['dividendYield'])

0.17889999


For each target stock get the PUT option chain with expiry dates more than 3 days but less than 15 days before the ex-div date ["exDividendDate"].

In [4]:
from datetime import datetime
tickers = []

for ticker in sorted_tickers:
    ticker_dates = {}
    ticker_dates["ticker"] = ticker
    ticker_dates["options"] = []
    exDividendDate = datetime.fromtimestamp(ticker.info["exDividendDate"]).date()
    ticker_dates["exDividendDate"] = exDividendDate
    for option in ticker.options:
        date = datetime.strptime(option, '%Y-%m-%d').date() 
        difference = exDividendDate - date
        if 3 < difference.days < 15:
            ticker_dates["options"].append(option)
    if len(ticker_dates["options"]) > 0:
        tickers.append(ticker_dates)

pprint(tickers)

[{'exDividendDate': datetime.date(2024, 1, 24),
  'options': ['2024-01-12', '2024-01-19'],
  'ticker': yfinance.Ticker object <PFE>},
 {'exDividendDate': datetime.date(2024, 2, 28),
  'options': ['2024-02-16'],
  'ticker': yfinance.Ticker object <TSN>},
 {'exDividendDate': datetime.date(2024, 1, 11),
  'options': ['2023-12-29', '2024-01-05'],
  'ticker': yfinance.Ticker object <HRL>},
 {'exDividendDate': datetime.date(2024, 1, 2),
  'options': ['2023-12-29'],
  'ticker': yfinance.Ticker object <CSCO>},
 {'exDividendDate': datetime.date(2024, 1, 15),
  'options': ['2024-01-05'],
  'ticker': yfinance.Ticker object <EOG>},
 {'exDividendDate': datetime.date(2024, 1, 3),
  'options': ['2023-12-29'],
  'ticker': yfinance.Ticker object <AEO>},
 {'exDividendDate': datetime.date(2024, 1, 16),
  'options': ['2024-01-05', '2024-01-12'],
  'ticker': yfinance.Ticker object <ACN>},
 {'exDividendDate': datetime.date(2024, 1, 10),
  'options': ['2023-12-29', '2024-01-05'],
  'ticker': yfinance.Ticker 

Remove all contracts with an open interest less than 1000 contracts.
Remove all contracts with a strike price greater than or less than 2% of the current market price.  

In [23]:
import pandas as pd

for ticker in tickers:
    for option in ticker["options"]:
        chain = ticker["ticker"].option_chain(option)
        puts_calls = pd.concat([chain.puts, chain.calls])
        res = puts_calls.query('`openInterest` > 1000')
    print(res)        

        contractSymbol             lastTradeDate  strike  lastPrice    bid  \
1   PFE240119P00017500 2023-12-19 14:30:11+00:00    17.5       0.01   0.00   
3   PFE240119P00020000 2023-12-21 16:36:00+00:00    20.0       0.05   0.00   
6   PFE240119P00022500 2023-12-22 20:55:47+00:00    22.5       0.01   0.00   
8   PFE240119P00024000 2023-12-26 16:38:38+00:00    24.0       0.04   0.02   
10  PFE240119P00025000 2023-12-26 15:52:27+00:00    25.0       0.04   0.04   
12  PFE240119P00026000 2023-12-26 17:37:24+00:00    26.0       0.10   0.09   
13  PFE240119P00026500 2023-12-26 16:40:00+00:00    26.5       0.15   0.15   
14  PFE240119P00027000 2023-12-26 17:33:55+00:00    27.0       0.24   0.23   
15  PFE240119P00027500 2023-12-26 17:38:03+00:00    27.5       0.37   0.37   
16  PFE240119P00028000 2023-12-26 17:39:22+00:00    28.0       0.56   0.56   
18  PFE240119P00029000 2023-12-26 17:38:58+00:00    29.0       1.12   1.10   
20  PFE240119P00030000 2023-12-26 17:09:41+00:00    30.0       1